In [ ]:
!pip install pandas requests

In [ ]:
import pandas as pd
import json
import requests

In [ ]:
def get_llm_response(prompt):
    url = "http://localhost:11434/api/generate"
    
    data = {
        "model": "llama3.3:70b-instruct-q8_0",
        "prompt": prompt,
        "stream": False
    }
    
    response = requests.post(url, json=data)
    return response.json()['response']

In [ ]:
!pip install requests
!pip install typing-extensions==4.7.1
!pip install pydantic==2.4.2
!pip install ollama

KSR Vol 1 - Chunk 2000, Overlap 300

KSR Vol 1 - Chunk 2000, Overlap 300

KSR

In [ ]:
import json
import requests
import re
from tqdm import tqdm

def extract_section_info(text):
    patterns = {
        'Document': r'Document:"([^"]+)"',
        'Part': r'Part:"([^"]+)"',
        'Chapter': r'Chapter:"([^"]+)"',
        'Appendix': r'Appendix:"([^"]+)"',
        'Annexure': r'Annexure:"([^"]+)"',
        'Section': r'Section:"([^"]+)"',
        'Sub Section': r'Sub Section:"([^"]+)"',
        'Sub division': r'Sub division:"([^"]+)"',
        'Rule no.': r'Rule no.:"([^"]+)"'
    }
    
    result = {k: None for k in patterns.keys()}
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            result[key] = match.group(1)
    
    return result

def process_text_chunk(text):
    # Split into sections based on "Rule no.:" pattern
    sections = re.split(r'(?=Rule no.:")', text)
    sections = [s for s in sections if s.strip()]

    results = []
    for section in sections:
        # Extract section info and description
        section_info = extract_section_info(section)
        
        # Extract description - everything after the last known field
        description_pattern = r'Description:"([^"]+)"'
        desc_match = re.search(description_pattern, section)
        if desc_match:
            section_info['Description'] = desc_match.group(1)
        
        if any(section_info.values()):
            results.append(section_info)
    
    return results

def chunk_text(text, max_length=2000, overlap=300):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + max_length
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap
    
    return chunks

def process_with_llama(text):
    prompt = """You are tasked with extracting structured data from a text block that contains information about rules or sections from a document, likely the Kerala Service Rules (KSR). Your goal is to parse this information and format it as a JSON list.

The JSON format for each entry should be as follows:
{
  "Document": "KSR",
  "Part": "",
  "Chapter": "",
  "Appendix": "",
  "Annexure": "",
  "Section": "",
  "Sub Section": "",
  "Sub division": "",
  "Rule no.": "",
  "Description": ""
}

Here is the text block you need to parse:

<text_block>
{?text}
</text_block>

To extract the required information:

1. Identify the document name (usually "KSR" unless specified otherwise).
2. Look for indicators of Part, Chapter, Appendix, Annexure, Section, Sub Section, Sub division, and Rule number. These may be explicitly stated or implied by the structure of the text.
3. The remaining text should be considered as the Description.
4. If any field is not present in the text, leave it as an empty string in the JSON.
5. If multiple rules or sections are present in the text block, create separate JSON objects for each.
6. Ensure that the Rule number is a string, even if it's a number.
7. For the Description, include all relevant text, including notes, government decisions, and examples if present.
8. Do not discard any part of the text block. If you cannot classify any portion of the text according to the specified fields (Part, Chapter, Appendix, Annexure, Section, Sub Section, Sub division, Rule number), include it in the Description field and populate its remaining attributes (Part, Chapter, Appendix, Annexure, Section, Sub Section, Sub division, Rule number) with the corresponding attributes from the previous JSON object, because these fields are mandatory. Do not put random stuff in these field, always put the correct ones. 
9. Format the Rule no. field exactly as shown in the template above.

Your output should be a valid JSON list containing one or more objects in the specified format. Do not include any additional text or explanations outside of the JSON structure.

<output>
</output>
""".replace("{?text}", text)

    response = requests.post('http://localhost:11434/api/generate',
                           json={
                               'model': 'llama3.3:70b-instruct-q8_0',
                               'prompt': prompt,
                               'stream': False
                           })
    
    if response.status_code == 200:
        try:
            response_json = response.json()['response']
            # Extract JSON content from markdown code block if present
            if '```json' in response_json:
                json_str = response_json.split('```json')[1].split('```')[0].strip()
            else:
                json_str = response_json.strip()
            
            # Parse the JSON string
            return json.loads(json_str)
        except Exception as e:
            print(f"Error processing LLM response: {e}")
            print(f"Response was: {response.json()['response']}")
            return None
    return None

def main():
    # Read the input file
#     with open('Extracted/KSR_Vol_2_pdfminer_extracted.txt', 'r', encoding='utf-8') as file:
#         text = file.read()
        
    # Open and read both files
    with open('Extracted/KSR_Vol_1_pdfminer_extracted.txt', 'r', encoding='utf-8') as file1, \
         open('Extracted/KSR_Vol_2_pdfminer_extracted.txt', 'r', encoding='utf-8') as file2:
        text = file1.read() + file2.read()
        
    
    # Process text in chunks
    chunks = chunk_text(text)  # Removed the [:3] slice to process all chunks
    all_results = []
    
    for chunk in tqdm(chunks, desc="Processing text chunks"):
        # Try direct pattern matching first
        results = process_text_chunk(chunk)
        
        # If pattern matching fails, use LLM
        if not results:
            llm_result = process_with_llama(chunk)
            if llm_result:
                all_results.extend(llm_result if isinstance(llm_result, list) else [llm_result])
        else:
            all_results.extend(results)
    
    # Add debug output
    print(f"Total results found: {len(all_results)}")
    
    # Write results to JSON file
    if all_results:
        with open('Full_KSR_extracted_rules2.json', 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=4, ensure_ascii=False)
        print("Results successfully written to Full_KSR_extracted_rules.json")
    else:
        print("No results were found to write to file")
    
    return all_results

if __name__ == "__main__":
    result = main()

Processing text chunks:   2%|█▋                                                                                                               | 10/659 [07:15<7:00:02, 38.83s/it]

KFC

In [ ]:
import json
import requests
import re
from tqdm import tqdm

def extract_section_info(text):
    patterns = {
        'Document': r'Document:"([^"]+)"',
        'Chapter': r'Chapter:"([^"]+)"',
        'Appendix': r'Appendix:"([^"]+)"',
        'Annexure': r'Annexure:"([^"]+)"',
        'Rule no.': r'Rule no.:"([^"]+)"'
    }
    
    result = {k: None for k in patterns.keys()}
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            result[key] = match.group(1)
    
    return result

def process_text_chunk(text):
    # Split into sections based on "Rule no.:" pattern
    sections = re.split(r'(?=Rule no.:")', text)
    sections = [s for s in sections if s.strip()]

    results = []
    for section in sections:
        # Extract section info and description
        section_info = extract_section_info(section)
        
        # Extract description - everything after the last known field
        description_pattern = r'Description:"([^"]+)"'
        desc_match = re.search(description_pattern, section)
        if desc_match:
            section_info['Description'] = desc_match.group(1)
        
        if any(section_info.values()):
            results.append(section_info)
    
    return results

def chunk_text(text, max_length=2000, overlap=300):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + max_length
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap
    
    return chunks

def process_with_llama(text):
    prompt = """You are tasked with extracting structured data from a text block that contains information about rules from a document, which is Kerala Financial Code Vol I. Your goal is to parse this information and format it as a JSON list.

The JSON format for each entry should be as follows:
{
  "Document": "KFC",
  "Chapter": "",
  "Appendix": "",
  "Annexure": "",
  "Rule no.": "",
  "Description": ""
}

Here is the text block you need to parse:

<text_block>
{?text}
</text_block>

To extract the required information:

1. Identify the document name (usually "KFC" unless specified otherwise).
2. Look for indicators of Chapter, Appendix, Annexure, and Rule number. These may be explicitly stated or implied by the structure of the text.
3. The remaining text should be considered as the Description.
4. If any field is not present in the text, leave it as an empty string in the JSON.
5. If multiple rules or sections are present in the text block, create separate JSON objects for each.
6. Ensure that the Rule number is a string, even if it's a number.
7. For the Description, include all relevant text, including notes, government decisions, and examples if present.
8. Do not discard any part of the text block. If you cannot classify any portion of the text according to the specified fields (Chapter, Appendix, Annexure, Rule no.), include it in the Description field.
9. Format the Rule no. field exactly as shown in the template above.

Your output should be a valid JSON list containing one or more objects in the specified format. Do not include any additional text or explanations outside of the JSON structure.

<output>
</output>
""".replace("{?text}", text)

    response = requests.post('http://localhost:11434/api/generate',
                           json={
                               'model': 'llama3.3:70b-instruct-q8_0',
                               'prompt': prompt,
                               'stream': False
                           })
    
    if response.status_code == 200:
        try:
            response_json = response.json()['response']
            # Extract JSON content from markdown code block if present
            # Clean up the response
            if '```json' in response_json:
                json_str = response_json.split('```json')[1].split('```')[0].strip()
            else:
                json_str = response_json.strip()
            
            # Remove any leading/trailing whitespace or newlines
            json_str = json_str.strip()
            
            # Parse the JSON string
            return json.loads(json_str)
        except Exception as e:
            print(f"Error processing LLM response: {e}")
            print(f"Response was: {response.json()['response']}")
            return None
    return None

def main():
    # Read the input file
    with open('/workspace/rohith_llm/Extracted/KFC1_pdfminer_extracted.txt', 'r', encoding='utf-8') as file:
        text = file.read()
        
            
    
    # Process text in chunks
    chunks = chunk_text(text)  # Removed the [:3] slice to process all chunks
    all_results = []
    
    for chunk in tqdm(chunks, desc="Processing text chunks"):
        # Try direct pattern matching first
        results = process_text_chunk(chunk)
        
        # If pattern matching fails, use LLM
        if not results:
            llm_result = process_with_llama(chunk)
            if llm_result:
                all_results.extend(llm_result if isinstance(llm_result, list) else [llm_result])
        else:
            all_results.extend(results)
    
    # Add debug output
    print(f"Total results found: {len(all_results)}")
    
    # Write results to JSON file
    if all_results:
        with open('Full_KFC_extracted_rules.json', 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=4, ensure_ascii=False)
        print("Results successfully written to Full_KFC_extracted_rules.json")
    else:
        print("No results were found to write to file")
    
    return all_results

if __name__ == "__main__":
    result = main()

KTC

In [25]:
import json
import requests
import re
from tqdm import tqdm

def extract_section_info(text):
    patterns = {
        'Document': r'Document:"([^"]+)"',
        'Part': r'Part:"([^"]+)"',
        'Section': r'Section:"([^"]+)"',
        'Rule no.': r'Rule no.:"([^"]+)"'
    }
    
    result = {k: None for k in patterns.keys()}
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            result[key] = match.group(1)
    
    return result

def process_text_chunk(text):
    # Split into sections based on "Rule no.:" pattern
    sections = re.split(r'(?=Rule no.:")', text)
    sections = [s for s in sections if s.strip()]

    results = []
    for section in sections:
        # Extract section info and description
        section_info = extract_section_info(section)
        
        # Extract description - everything after the last known field
        description_pattern = r'Description:"([^"]+)"'
        desc_match = re.search(description_pattern, section)
        if desc_match:
            section_info['Description'] = desc_match.group(1)
        
        if any(section_info.values()):
            results.append(section_info)
    
    return results

def chunk_text(text, max_length=2000, overlap=300):
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + max_length
        chunk = text[start:end]
        chunks.append(chunk)
        start = end - overlap
    
    return chunks

def process_with_llama(text):
    prompt = """You are tasked with extracting structured data from a text block that contains information about rules from a document, which is Kerala Financial Code Vol I. Your goal is to parse this information and format it as a JSON list.

The JSON format for each entry should be as follows:
{
  "Document": "KTC",
  "Part": "", #always present
  "Section": "", #present most of the time
  "Rule no.": "", #always present
  "Description": ""
}

Here is the text block you need to parse:

<text_block>
{?text}
</text_block>

To extract the required information:

1. Identify the document name (usually "KTC" unless specified otherwise).
2. Look for indicators of Part, Section, and Rule number. These may be explicitly stated or implied by the structure of the text.
3. The remaining text should be considered as the Description.
4. If any field is not present in the text, leave it as an empty string in the JSON.
5. If multiple rules or sections are present in the text block, create separate JSON objects for each.
6. Ensure that the Rule number is a string, even if it's a number.
7. For the Description, include all relevant text, including notes, government decisions, and examples if present.
8. Do not discard any part of the text block. If you cannot classify any portion of the text according to the specified fields (Part, Section, Rule number), include it in the Description field and populate its remaining attributes (Part, Section, and Rule number) with the corresponding attributes from the previous JSON object, because these fields are mandatory. Do not put random stuff in these field, always put the correct ones. 
9. Format the Rule no. field exactly as shown in the template above.

Your output should be a valid JSON list containing one or more objects in the specified format. Do not include any additional text or explanations outside of the JSON structure.

<output>
</output>
""".replace("{?text}", text)

    response = requests.post('http://localhost:11434/api/generate',
                           json={
                               'model': 'llama3.3:70b-instruct-q8_0',
                               'prompt': prompt,
                               'stream': False
                           })
    
    if response.status_code == 200:
        try:
            response_json = response.json()['response']
            # Extract JSON content from markdown code block if present
            # Clean up the response
            if '```json' in response_json:
                json_str = response_json.split('```json')[1].split('```')[0].strip()
            else:
                json_str = response_json.strip()
            
            # Remove any leading/trailing whitespace or newlines
            json_str = json_str.strip()
            
            # Parse the JSON string
            return json.loads(json_str)
        except Exception as e:
            print(f"Error processing LLM response: {e}")
            print(f"Response was: {response.json()['response']}")
            return None
    return None

def main():
    # Read the input file
    with open('/workspace/rohith_llm/Extracted/KTC_pdfminer_extracted.txt', 'r', encoding='utf-8') as file:
        text = file.read()
        
            
    
    # Process text in chunks
    chunks = chunk_text(text)  # Removed the [:3] slice to process all chunks
    all_results = []
    
    for chunk in tqdm(chunks, desc="Processing text chunks"):
        # Try direct pattern matching first
        results = process_text_chunk(chunk)
        
        # If pattern matching fails, use LLM
        if not results:
            llm_result = process_with_llama(chunk)
            if llm_result:
                all_results.extend(llm_result if isinstance(llm_result, list) else [llm_result])
        else:
            all_results.extend(results)
    
    # Add debug output
    print(f"Total results found: {len(all_results)}")
    
    # Write results to JSON file
    if all_results:
        with open('Full_KTC_extracted_rules.json', 'w', encoding='utf-8') as f:
            json.dump(all_results, f, indent=4, ensure_ascii=False)
        print("Results successfully written to Full_KTC_extracted_rules.json")
    else:
        print("No results were found to write to file")
    
    return all_results

if __name__ == "__main__":
    result = main()

Processing text chunks:   0%|▏                                                                                                                 | 1/507 [00:39<5:37:14, 39.99s/it]

Error processing LLM response: Expecting ':' delimiter: line 6 column 18 (char 83)
Response was: ```json
[
  {
    "Document": "KTC",
    "Part": "I",
    "Section": "I",
    "Rule no.': "",
    "Description": "Short title and Commencement"
  },
  {
    "Document": "KTC",
    "Part": "I",
    "Section": "II",
    "Rule no.': "",
    "Description": "Definitions"
  },
  {
    "Document": "KTC",
    "Part": "I",
    "Section": "III",
    "Rule no.': "",
    "Description": "Location of moneys standing in the Government Account"
  },
  {
    "Document": "KTC",
    "Part": "I",
    "Section": "IV",
    "Rule no.': "",
    "Description": "General system of control over Treasury— District Treasuries, Sub Treasuries"
  },
  {
    "Document": "KTC",
    "Part": "I",
    "Section": "V",
    "Rule no.': "",
    "Description": "Payment of moneys into the Government Account"
  },
  {
    "Document": "KTC",
    "Part": "I",
    "Section": "VI",
    "Rule no.': "",
    "Description": "Custody of money

Processing text chunks:   0%|▍                                                                                                                 | 2/507 [01:14<5:10:45, 36.92s/it]

Error processing LLM response: Expecting ':' delimiter: line 6 column 18 (char 83)
Response was: ```json
[
  {
    "Document": "KTC",
    "Part": "II",
    "Section": "",
    "Rule no.': "",
    "Description": "GENERAL  ORGANISATION AND  WORKING  OF    THE  TREASURIES"
  },
  {
    "Document": "KTC",
    "Part": "II",
    "Section": "I",
    "Rule no.': "",
    "Description": "District Treasuries: Responsibilities of the Director Responsibilities of the Treasury Officer Responsibilities of the Chief Treasurer Verification of treasury balance when there is a change of  Treasury Officer or Chief Treasurer Responsibility of the Section Heads in the Accounts Department"
  },
  {
    "Document": "KTC",
    "Part": "II",
    "Section": "I",
    "Rule no.': "",
    "Description": "Sub Treasuries.— Location of Sub Treasuries Sub Treasury Officer Other employees of the Sub Treasury Inspection of Sub Treasuries Banks conducting treasury business"
  },
  {
    "Document": "KTC",
    "Part": "II",

Processing text chunks:   1%|█▎                                                                                                                | 6/507 [05:38<9:03:26, 65.08s/it]

...

Error processing LLM response: Expecting ':' delimiter: line 6 column 18 (char 81)
Response was: ```json
[
  {
    "Document": "KTC",
    "Part": "",
    "Section": "",
    "Rule no.': "",
    "Description": "s recorded by the disbursing officer against the entry recording the payment.** Omitted [G.O.(P) 347/81/Fin., dated 28th May, 1981] 348 KERALA TREASURY CODE GOVERNMENT OF KERALA Abstract DISBURSEMENT OF PERSONAL ENTITLEMENTS OF EMPLOYEES INCLUDING SELF DRAWING OFFICERS OF ALL GOVERNMENT DEPARTMENTS THROUGH BANKS- SANCTIONED—ORDERS ISSUED FINANCE (STREAMLINING) DEPARTMENT G. O. (P) No. 402/11/Fin. Dated, Thiruvananthapuram, 23-9-2011 Read:— 1. G. O. (P) No. 439/09/Fin dated 12-10-2009. 2. G. O. (P) No. 476/09/Fin dated 2-11-2009. 3. Circular No. 14/2011/Fin dated 17-2-2011 4. G. O. (P) No. 658/10/Fin dated 3-12-2010."
  },
  {
    "Document": "KTC",
    "Part": "",
    "Section": "",
    "Rule no.': "",
    "Description": "As per Government Order 1st read above, sanction was accord

Processing text chunks: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 506/507 [4:07:21<00:29, 29.42s/it]

Error processing LLM response: Expecting ':' delimiter: line 6 column 18 (char 81)
Response was: ```json
[
  {
    "Document": "KTC",
    "Part": "",
    "Section": "",
    "Rule no.': "163 (k)",
    "Description": "Recoveries from the salary of government servants on account of dues to co-operative societies registered under the various Co-operative Societies Acts where such Acts impose a statutory obligation on the Government to make such deductions shall be made by the drawing and disbursing officers or the authorized banks  in the case of Non-Gazetted Government servants who do not draw their own bills, and in other cases by the Treasury Officers or the authorized banks/other disbursing officers concerned, as the case may be, in accordance with such procedure as may be laid down by Government from time to time"
  },
  {
    "Document": "KTC",
    "Part": "",
    "Section": "",
    "Rule no.': "432(a)",
    "Description": "The head of an office is personally responsible for all mone

Processing text chunks: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 507/507 [4:07:48<00:00, 29.33s/it]

Total results found: 1071
Results successfully written to Full_KTC_extracted_rules.json
